# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
# importing packages
import numpy as np 
import pandas as pd
from sklearn.cluster import KMeans


In [2]:
# importing pakage for web scraping
!pip install beautifulSoup4 
!pip install lxml # import package for reading xml 
!pip install requests #Requests allows you to send organic, grass-fed HTTP/1.1 requests,

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


parsers to parse html

In [5]:
#  import the installed packages
from bs4 import BeautifulSoup
import requests 

In [6]:
# read the data
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [7]:
soup=BeautifulSoup(website_url,'lxml')

In [8]:
# to see the data clearly with indentation and tags
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 errors: deprecated parameters","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","Feb

In [9]:
# this is found in the above output as the place where the table is created in the webpage.. 
# to access the table use the class and name of the class
table=soup.find('table',{'class':'wikitable sortable'})

In [10]:
print(table) # view the table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [11]:
links=table.find_all('a')#Now to extract all the links within <a>, we will use find_all().
post=table.find_all('tr') #similarly for tr

In [12]:
# this code creates the df with columns and rows fetched from the table..    
    
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
#             for col in df:
#                 try:
#                     df[col] = df[col].astype(float)
#                 except ValueError:
#                     pass

In [13]:
# removing the newline added in the column name
df['Neighbourhood']=df['Neighbourhood\n']

In [224]:
# remove the \n from data for the column as well
for i in df['Neighbourhood']:
    df['Neighbourhood']=df.Neighbourhood.str.replace('\n','')


In [16]:
df=df[df['Borough']!='Not assigned']  #remove all not assigned cells from column

In [17]:
# find rows where neighbourhood is not assigned
idx_to_change=df.loc[df['Neighbourhood']=='Not assigned']
# print(idx_to_change['Neighbourhood'])

# rename the not assigned go borough column value
for i in idx_to_change:
    idx_to_change['Neighbourhood']=idx_to_change['Borough']

# compare neighbourhood\n(old column) with neighbourhood(new column with Borough cell value)
print(idx_to_change)


    

  Postcode       Borough Neighbourhood\n Neighbourhood
8      M7A  Queen's Park  Not assigned\n  Queen's Park


C:\Users\DR36495\AnacondaWithSpyder\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [18]:
df.drop(['Neighbourhood\n'],axis=1,inplace=True)# this column was an extra column so removing it


In [20]:
# df1=df.groupby(['Postcode','Borough']).apply(lambda x:','.join(x.Neighbourhood)) This does not work but good for 
df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ','.join(x))

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Not assigned
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge,Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens,Parkview Hill"


In [181]:
df.shape

(211, 5)

In [22]:
!pip install geocoder

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [23]:
import geocoder
# !pip install geopy
from geopy.geocoders import Nominatim

In [24]:
from pandas.io.json import json_normalize 

In [148]:
# check lon lan for one borough
address = 'Scarborough	'

geolocator = Nominatim(user_agent="Canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 54.2820009, -0.4011868.


In [26]:
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [182]:
import folium
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [150]:
# df['latitude']=None
# df['longitude']=None
# for i,rows in df.iterrows():
#     lat_lan=None
#     while (lat_lan==None):
#         g=geocoder.arcgis(rows['Borough'].format(df['Postcode']))
#         lat_lan=g.x+g.y
#     df.loc[i,'latitude']=g.y
#     df.loc[i,'longitude']=g.x


In [177]:
# df.reset_index(drop=True)
# # df.head()
df['latitude']=None
df['longitude']=None
for i,j in df.iterrows():
    address=j['Borough']
    geolocator = Nominatim(user_agent="Canada_explorer")
    
    location = geolocator.geocode(address)
    df.loc[i,'latitude'] = location.latitude
    df.loc[i,'longitude'] = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of Downtown Toronto are 43.7708175, -79.4132998.
The geograpical coordinate of Downtown Toronto are 43.7708175, -79.4132998.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of Queen's Park are 43.7708175, -79.4132998.
The geograpical coordinate of Etobicoke are 43.7708175, -79.4132998.
The geograpical coordinate of Scarborough are 43.7708175, -79.4132998.
The geograpical coordinate of Scarborough are 43.7708175, -79.4132998.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of East York are 43.7708175, -79.4132998.
The geograpical coordinate of East York are 43.7708175, -79.4132998.
The geograpical coordinate of Downtown Toronto are 43.7708175, -79.4132998.
T

The geograpical coordinate of York are 43.7708175, -79.4132998.
The geograpical coordinate of Scarborough are 43.7708175, -79.4132998.
The geograpical coordinate of Scarborough are 43.7708175, -79.4132998.
The geograpical coordinate of Scarborough are 43.7708175, -79.4132998.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of Central Toronto are 43.7708175, -79.4132998.
The geograpical coordinate of Central Toronto are 43.7708175, -79.4132998.
The geograpical coordinate of Central Toronto are 43.7708175, -79.4132998.
The geograpical coordinate of West Toronto are 43.7708175, -79.4132998.
The geograpical coordinate of West Toronto are 43.7708175, -79.4132998.
The geograpical coordinate of Etobicoke are 43.7708175, -79.4132998.
The geograpical coordinate of Scarborough are 43.7708175, -79.4132998.
The geograpical coordinate of Scarborough are 43.7708175, -79.4132998.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The

In [178]:
!pip install folium

df.head()

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


,Postcode,Borough,Neighbourhood,latitude,longitude
2,M3A,North York,Parkwoods,43.7708,-79.4133
3,M4A,North York,Victoria Village,43.7708,-79.4133
4,M5A,Downtown Toronto,Harbourfront,43.6542,-79.3808
5,M5A,Downtown Toronto,Regent Park,43.6542,-79.3808
6,M6A,North York,Lawrence Heights,43.7708,-79.4133


In [41]:
import folium
# df=df[df['Borough'].str.contains('Toronto')]


In [180]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 211 neighborhoods.


In [183]:
# create map of Toronto Map using latitude and longitude values

map_ny=folium.Map(location=[-80.73502252,  43.65011 ],zoom_start=10)

# add markers to the map
for lat, lon, borough, neighbor in zip(df['latitude'],df['longitude'],df['Borough'],df['Neighbourhood']):
#     print(lat, lon,borough)
    label='{},{}'.format(neighbor,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny
    
    

In [45]:
CLIENT_ID='IUQMV40FTE1IMIAEOBV1SDMIFYTLV242NGKDWYVYKTCMGJVB'

CLIENT_SECRET='0EH4DMV3I5SY3M5GA51KZOCI4DBGGCQWIWY4IIUXMZY3PMV2'
VERSION='20190916'
LIMIT =100



In [47]:
neighborhood_latitude = df.loc[265,'latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[265,'longitude'] # neighborhood longitude value

neighborhood_name = df.loc[265, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                            
                                                               neighborhood_longitude))
LIMIT = 100 
radius = 1000

Latitude and longitude values of Business Reply Mail Processing Centre 969 Eastern are 40.47834122154802, -80.73502252176193.


In [48]:


url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()
# results = requests.get('https://mytestproject.com').json()

normal_results=json_normalize(results)

normal_results.columns


Index(['meta.code', 'meta.requestId', 'response.venues', 'response.confident'], dtype='object')

In [185]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [186]:

from pandas.io.json import json_normalize

In [187]:
venues = results['response']['venues'][0]


nearby_venues=json_normalize(venues)#flatten JSON

print(nearby_venues.columns)
    
# # filter columns
filtered_columns = ['name','categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# # filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
print(nearby_venues)
# # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.lat',
       'location.lng', 'location.labeledLatLngs', 'location.distance',
       'location.postalCode', 'location.cc', 'location.city', 'location.state',
       'location.country', 'location.formattedAddress'],
      dtype='object')
          name                                         categories  \
0  Austin Lake  [{'id': '4bf58dd8d48988d1e4941735', 'name': 'C...   

   location.lat  location.lng venue.categories  
0     40.493996    -80.743422       Campground  


,name,categories,lat,lng,categories
0,Austin Lake,"[{'id': '4bf58dd8d48988d1e4941735', 'name': 'C...",40.493996,-80.743422,Campground


In [52]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


# Part 2 Explore neighborhood for Etobicoke Borough

In [188]:
# # create df for etobicoke called as Etobicoke
# Etobicoke = df[df['Borough'].isin(['Downtown Toronto','East Toronto','West Toronto','Central Toronto'])==TRUE].reset_index(drop=True)
# Etobicoke.head()

Etobicoke=df.reset_index(drop=True)
Etobicoke.head()
# df['Borough'].unique()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7708,-79.4133
1,M4A,North York,Victoria Village,43.7708,-79.4133
2,M5A,Downtown Toronto,Harbourfront,43.6542,-79.3808
3,M5A,Downtown Toronto,Regent Park,43.6542,-79.3808
4,M6A,North York,Lawrence Heights,43.7708,-79.4133


Create a function to to repeat the same process as above to all neighbourrhood for etobicoke

In [189]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
 # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
       

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now is the code to call the above function for each neighbourhood call the new datafame all_venues

In [190]:
# type your answer here

all_venues = getNearbyVenues(names=Etobicoke['Neighbourhood'],
                                   latitudes=Etobicoke['latitude'],
                                   longitudes=Etobicoke['longitude']
                                  )



Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Not assigned
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design E

In [191]:
# check the size of all_venues
# check head of all_vernues
print(all_venues.shape)
all_venues.head()

(11855, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.770817,-79.4133,The Captain's Boil,43.773255,-79.413805,Seafood Restaurant
1,Parkwoods,43.770817,-79.4133,Aroma Espresso Bar,43.769449,-79.413081,Café
2,Parkwoods,43.770817,-79.4133,Starbucks,43.768353,-79.413046,Coffee Shop
3,Parkwoods,43.770817,-79.4133,Loblaws,43.768722,-79.412101,Grocery Store
4,Parkwoods,43.770817,-79.4133,Konjiki Ramen,43.766998,-79.412222,Ramen Restaurant


# Analyse each neighbourhood

In [192]:
# check number of unique neighbourhoods
all_venues['Neighborhood'].unique()
all_venues.columns

Index(['Neighborhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [193]:
#  one hot encoding creating dummies 
Etobicoke_onehot=pd.get_dummies(all_venues[['Venue Category']], prefix="",prefix_sep="")
print(Etobicoke_onehot)

       American Restaurant  Arcade  Art Gallery  Art Museum  \
0                        0       0            0           0   
1                        0       0            0           0   
2                        0       0            0           0   
3                        0       0            0           0   
4                        0       0            0           0   
...                    ...     ...          ...         ...   
11850                    0       0            0           0   
11851                    0       0            0           0   
11852                    0       0            0           0   
11853                    0       0            0           0   
11854                    0       0            0           0   

       Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Bakery  \
0                        0                 0                   0       0   
1                        0                 0                   0       0   
2              

In [194]:
# adding neighbourhood column back to this new dataframe
Etobicoke_onehot['Neighbourhood']=all_venues['Neighborhood']
# move neighbourhood column to the first column

fixed_columns = [Etobicoke_onehot.columns[-1]] + list(Etobicoke_onehot.columns[:-1])
Etobicoke_onehot=Etobicoke_onehot[fixed_columns]
# check head
Etobicoke_onehot.head()


,Neighbourhood,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [195]:
# Examine th size of neew dataframe
Etobicoke_onehot.shape

(11855, 140)

In [196]:
# lets lets group rows by neighbourhood and take mean of the freq of occurance of each category
Etobicoke_grouped=Etobicoke_onehot.groupby('Neighbourhood').mean().reset_index()
Etobicoke_grouped.head()

,Neighbourhood,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.03,0.000000,0.0,0.0,0.0,0.01,0.0,0.03,0.01,...,0.0,0.0,0.01,0.010000,0.0,0.000000,0.0,0.0,0.01,0.0
1,Agincourt,0.00,0.038462,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,0.0,0.00,0.019231,0.0,0.019231,0.0,0.0,0.00,0.0
2,Agincourt North,0.00,0.038462,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,0.0,0.00,0.019231,0.0,0.019231,0.0,0.0,0.00,0.0
3,Albion Gardens,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0
4,Alderwood,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0


In [197]:
# lets print each neighbourhood and the 5 most common venues

num_top_venues=5

for hood in Etobicoke_grouped['Neighbourhood']:
    print('*****'+hood+'*****')
    temp=Etobicoke_grouped[Etobicoke_grouped['Neighbourhood']==hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    

*****Adelaide*****
                 venue  freq
0       Clothing Store  0.08
1          Coffee Shop  0.05
2           Restaurant  0.04
3  American Restaurant  0.03
4               Bakery  0.03


*****Agincourt*****
            venue  freq
0  Clothing Store  0.08
1            Café  0.08
2     Pizza Place  0.06
3           Hotel  0.06
4     Coffee Shop  0.06


*****Agincourt North*****
            venue  freq
0  Clothing Store  0.08
1            Café  0.08
2     Pizza Place  0.06
3           Hotel  0.06
4     Coffee Shop  0.06


*****Albion Gardens*****
         venue  freq
0     Pharmacy   0.2
1  Supermarket   0.2
2       Garden   0.2
3  Coffee Shop   0.2
4   Playground   0.2


*****Alderwood*****
         venue  freq
0     Pharmacy   0.2
1  Supermarket   0.2
2       Garden   0.2
3  Coffee Shop   0.2
4   Playground   0.2


*****Bathurst Manor*****
                 venue  freq
0          Coffee Shop  0.09
1     Ramen Restaurant  0.08
2    Korean Restaurant  0.04
3          Pizza Place  0

         venue  freq
0     Pharmacy   0.2
1  Supermarket   0.2
2       Garden   0.2
3  Coffee Shop   0.2
4   Playground   0.2


*****Exhibition Place*****
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.04
2          Art Gallery  0.04
3       Breakfast Spot  0.04
4  Japanese Restaurant  0.04


*****Fairview*****
                 venue  freq
0          Coffee Shop  0.09
1     Ramen Restaurant  0.08
2    Korean Restaurant  0.04
3          Pizza Place  0.04
4  Japanese Restaurant  0.04


*****First Canadian Place*****
                 venue  freq
0       Clothing Store  0.08
1          Coffee Shop  0.05
2           Restaurant  0.04
3  American Restaurant  0.03
4               Bakery  0.03


*****Flemingdon Park*****
                 venue  freq
0          Coffee Shop  0.09
1     Ramen Restaurant  0.08
2    Korean Restaurant  0.04
3          Pizza Place  0.04
4  Japanese Restaurant  0.04


*****Forest Hill North*****
                 venue  freq
0       

4   Playground   0.2


*****Mimico South*****
         venue  freq
0     Pharmacy   0.2
1  Supermarket   0.2
2       Garden   0.2
3  Coffee Shop   0.2
4   Playground   0.2


*****Montgomery Road*****
         venue  freq
0     Pharmacy   0.2
1  Supermarket   0.2
2       Garden   0.2
3  Coffee Shop   0.2
4   Playground   0.2


*****Moore Park*****
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.04
2          Art Gallery  0.04
3       Breakfast Spot  0.04
4  Japanese Restaurant  0.04


*****Morningside*****
            venue  freq
0  Clothing Store  0.08
1            Café  0.08
2     Pizza Place  0.06
3           Hotel  0.06
4     Coffee Shop  0.06


*****Mount Dennis*****
                venue  freq
0                 Pub  0.16
1                Café  0.08
2                 Bar  0.05
3        Cocktail Bar  0.04
4  Italian Restaurant  0.04


*****Mount Olive*****
         venue  freq
0     Pharmacy   0.2
1  Supermarket   0.2
2       Garden   0.2
3  Coffe

4  Japanese Restaurant  0.04


*****Sunnylea*****
         venue  freq
0     Pharmacy   0.2
1  Supermarket   0.2
2       Garden   0.2
3  Coffee Shop   0.2
4   Playground   0.2


*****Swansea*****
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.04
2          Art Gallery  0.04
3       Breakfast Spot  0.04
4  Japanese Restaurant  0.04


*****Tam O'Shanter*****
            venue  freq
0  Clothing Store  0.08
1            Café  0.08
2     Pizza Place  0.06
3           Hotel  0.06
4     Coffee Shop  0.06


*****The Annex*****
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.04
2          Art Gallery  0.04
3       Breakfast Spot  0.04
4  Japanese Restaurant  0.04


*****The Beaches*****
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.04
2          Art Gallery  0.04
3       Breakfast Spot  0.04
4  Japanese Restaurant  0.04


*****The Beaches West*****
                 venue  freq
0       

Putting the above in a pandas dataframe
Function to sort the venues in descending order.

In [198]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [199]:
# Now let's create the new dataframe and display the top 5 venues for each neighborhood.

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Etobicoke_grouped['Neighbourhood']

for ind in np.arange(Etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()



,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adelaide,Clothing Store,Coffee Shop,Restaurant,American Restaurant,Bakery
1,Agincourt,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
2,Agincourt North,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
3,Albion Gardens,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
4,Alderwood,Supermarket,Garden,Playground,Coffee Shop,Pharmacy


# Clustering Neighbours

In [200]:
#  run k-means to cluster the neighbours
# set number of clusters
kclusters = 5

Etobicoke_grouped_clustering1 = Etobicoke_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Etobicoke_grouped_clustering1)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5]

array([4, 2, 2, 1, 1])

In [201]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Etobicoke_merged = Etobicoke

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Etobicoke_merged = Etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Etobicoke_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.7708,-79.4133,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
1,M4A,North York,Victoria Village,43.7708,-79.4133,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.6542,-79.3808,4,Clothing Store,Coffee Shop,Restaurant,American Restaurant,Bakery
3,M5A,Downtown Toronto,Regent Park,43.6542,-79.3808,4,Clothing Store,Coffee Shop,Restaurant,American Restaurant,Bakery
4,M6A,North York,Lawrence Heights,43.7708,-79.4133,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant


In [202]:
import matplotlib.cm as cm

import matplotlib.colors as colors
map_clusters= folium.Map(location=[latitude,longitude],zoom_start=11)

# set color scheme for the clusters
x=np.arange(kclusters)
ys=[i+x+(i+x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

#  add markers to the map


markers_color=[]
for lat, lon,poi,cluster in zip(Etobicoke_merged['latitude'],Etobicoke_merged['longitude'],Etobicoke_merged['Neighbourhood'],Etobicoke_merged['Cluster Labels']):
    label=folium.Popup(str(poi)+'Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7).add_to(map_clusters
#     http://localhost:8888/notebooks/Capstone%20w3%20Part2%20.ipynb#
        
        
map_clusters                       

Index(['Postcode', 'Borough', 'Neighbourhood', 'latitude', 'longitude'], dtype='object')

# Examine the Clusters individually

In [214]:
Etobicoke_merged.head()

,Postcode,Borough,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.7708,-79.4133,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
1,M4A,North York,Victoria Village,43.7708,-79.4133,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.6542,-79.3808,4,Clothing Store,Coffee Shop,Restaurant,American Restaurant,Bakery
3,M5A,Downtown Toronto,Regent Park,43.6542,-79.3808,4,Clothing Store,Coffee Shop,Restaurant,American Restaurant,Bakery
4,M6A,North York,Lawrence Heights,43.7708,-79.4133,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant


In [218]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 0, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
28,York,0,Pub,Café,Bar,Cocktail Bar,Italian Restaurant
36,East Toronto,0,Coffee Shop,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant
38,York,0,Pub,Café,Bar,Cocktail Bar,Italian Restaurant
52,West Toronto,0,Coffee Shop,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant
53,West Toronto,0,Coffee Shop,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant
64,West Toronto,0,Coffee Shop,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant
65,West Toronto,0,Coffee Shop,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant
72,East Toronto,0,Coffee Shop,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant
73,East Toronto,0,Coffee Shop,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant
76,West Toronto,0,Coffee Shop,Breakfast Spot,Art Gallery,Japanese Restaurant,Sushi Restaurant


In [220]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels']==1,Etobicoke_merged.columns[[1]+list(range(5,Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
16,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
17,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
18,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
19,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
20,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
29,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
30,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
31,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy
32,Etobicoke,1,Supermarket,Garden,Playground,Coffee Shop,Pharmacy


In [221]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels']==2,Etobicoke_merged.columns[[1]+list(range(5,Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
9,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
21,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
22,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
23,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
33,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
34,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
35,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
39,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
43,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach


In [222]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels']==3,Etobicoke_merged.columns[[1]+list(range(5,Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
1,North York,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
4,North York,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
5,North York,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
6,Queen's Park,3,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Park
10,North York,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
11,East York,3,Coffee Shop,Farmers Market,Pizza Place,Park,Athletics & Sports
12,East York,3,Coffee Shop,Farmers Market,Pizza Place,Park,Athletics & Sports
15,North York,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant
24,North York,3,Coffee Shop,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant


In [223]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels']==5,Etobicoke_merged.columns[[1]+list(range(5,Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
